In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import pandas as pd
import numpy as np
df = pd.read_csv('/kaggle/input/text-classification-data/training.csv')
df.head()

In [ ]:
import matplotlib.pyplot as plt
df.groupby('topic').article_words.count().plot.bar(ylim=0)
plt.show()

In [ ]:
# Lowercasing the text
df['article_words'] = df['article_words'].str.lower()
df['article_words'] = df['article_words'].str.replace(',', ' ')

In [ ]:
category_codes = {
    'ARTS CULTURE ENTERTAINMENT': 0,
    'BIOGRAPHIES PERSONALITIES PEOPLE': 1,
    'DEFENCE': 2,
    'DOMESTIC MARKETS': 3,
    'FOREX MARKETS': 4,
    'HEALTH': 5,
    'MONEY MARKETS': 6,
    'SCIENCE AND TECHNOLOGY': 7,
    'SHARE LISTINGS': 8,
    'SPORTS': 9,
    'IRRELEVANT': 10
}

In [ ]:
# Category mapping
df['topic_code'] = df['topic']
df = df.replace({'topic_code':category_codes})
df.head()

In [ ]:
X_train = df['article_words'][:9000]
y_train = df['topic_code'][:9000]
X_test = df['article_words'][9000:]
y_test = df['topic_code'][9000:]

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
# Parameter election
ngram_range = (1,2)
min_df = 10
max_df = 1.
max_features = 300
tfidf = TfidfVectorizer(encoding='utf-8',
                        ngram_range=ngram_range,
                        stop_words=None,
                        lowercase=False,
                        max_df=max_df,
                        min_df=min_df,
                        max_features=max_features,
                        norm='l2',
                        sublinear_tf=True)
#fit and transform - training set                        
features_train = tfidf.fit_transform(X_train).toarray()
labels_train = y_train
print(features_train.shape)

#transform test set
features_test = tfidf.transform(X_test).toarray()
labels_test = y_test
print(features_test.shape)

In [ ]:
labels_train

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from pprint import pprint
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import ShuffleSplit

In [ ]:
#Cross-Validation for Hyperparameter tuning
knnc_0 =KNeighborsClassifier()

print('Parameters currently in use:\n')
pprint(knnc_0.get_params())

In [ ]:
# Create the parameter grid 
n_neighbors = [int(x) for x in np.linspace(start = 1, stop = 500, num = 100)]

param_grid = {'n_neighbors': n_neighbors}

# Create a base model
knnc = KNeighborsClassifier()

# Manually create the splits in CV in order to be able to fix a random_state (GridSearchCV doesn't have that argument)
cv_sets = ShuffleSplit(n_splits = 3, test_size = .33, random_state = 8)

# Instantiate the grid search model
grid_search = GridSearchCV(estimator=knnc, 
                           param_grid=param_grid,
                           scoring='accuracy',
                           cv=cv_sets,
                           verbose=1)

# Fit the grid search to the data
grid_search.fit(features_train, labels_train)